# `JAXsim` Showcase: Rigid Body Dynamics

First, we install the necessary packages and import them.

<a target="_blank" href="https://colab.research.google.com/github/ami-iit/jaxsim/blob/main/examples/PD_controller.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# @title Imports and setup
from IPython.display import clear_output
import os
import pathlib
import sys
import robot_descriptions

IS_COLAB = "google.colab" in sys.modules

# Install JAX and Gazebo SDF
if IS_COLAB:
    !{sys.executable} -m pip install -qU jaxsim
    !apt install -qq lsb-release wget gnupg
    !wget https://packages.osrfoundation.org/gazebo.gpg -O /usr/share/keyrings/pkgs-osrf-archive-keyring.gpg
    !echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/pkgs-osrf-archive-keyring.gpg] http://packages.osrfoundation.org/gazebo/ubuntu-stable $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/gazebo-stable.list > /dev/null
    !apt -qq update
    !apt install -qq --no-install-recommends libsdformat13 gz-tools2

    clear_output()


import jax
import jax.numpy as jnp
from jaxsim import logging

logging.set_logging_level(logging.LoggingLevel.INFO)
logging.info(f"Running on {jax.devices()}")

We will use a model of the [ErgoCub](https://ergocub.eu/) humanoid robot.

In [ ]:
# @title Fetch the URDF file
try:
    os.environ["ROBOT_DESCRIPTION_COMMIT"] = "v0.7.1"

    import robot_descriptions.ergocub_description

finally:
    _ = os.environ.pop("ROBOT_DESCRIPTION_COMMIT", None)

model_urdf_path = pathlib.Path(
    robot_descriptions.ergocub_description.URDF_PATH.replace(
        "ergoCubSN000", "ergoCubSN001"
    )
)

JAXsim offers a simple functional API in order to interact in a memory-efficient way with the simulation. Four main elements are used to define a simulation:

- `model`: an object that defines the dynamics of the system.
- `data`: an object that contains the state of the system.
- `integrator`: an object that defines the integration method.
- `integrator_state`: an object that contains the state of the integrator.

The last two elements are not going to be used in this example.

In [ ]:
import jaxsim.api as js

# Create the model from the URDF file.
model = js.model.JaxSimModel.build_from_model_description(
    model_description=model_urdf_path
)

It is possible to reduce the model by lumping some links together in order to simplify the computation. 

In [ ]:
model = js.model.reduce(
    model=model,
    considered_joints=tuple(
        j
        for j in model.joint_names()
        if "camera" not in j
        # Remove head and hands.
        and "neck" not in j
        and "wrist" not in j
        and "thumb" not in j
        and "index" not in j
        and "middle" not in j
        and "ring" not in j
        and "pinkie" not in j
        # Remove upper body.
        and "torso" not in j and "elbow" not in j and "shoulder" not in j
    ),
)

In [ ]:
# Create the data object from the reduced model.
data = js.data.random_model_data(model=model)

Let's now create a random `references` object. This will be useful to manage the link forces and the torques applied to the joints.

In [ ]:
# Create a random link forces matrix.
link_forces = jax.random.uniform(
    minval=-10.0,
    maxval=10.0,
    shape=(model.number_of_links(), 6),
    key=jax.random.PRNGKey(0),
)

# Create a random joint forces references vector.
joint_force_references = jax.random.uniform(
    minval=-10.0, maxval=10.0, shape=(model.dofs(),), key=jax.random.PRNGKey(0)
)

# Create the references object.
references = js.references.JaxSimModelReferences.build(
    model=model,
    data=data,
    link_forces=link_forces,
    joint_force_references=joint_force_references,
)

**Note:**
- The argument `joint_force_references` is called `joint_force_references` because the actual joint forces will take into account the effect of the joint frictions.

## Forward Dynamics

We can compute the forward dynamics of the model by inverting the free floating mass matrix, e.g. starting from:

$$
M(q) \dot{\nu} + h(q, \nu) = B\tau + J^T(q) f
$$

we can compute the acceleration $\dot{\nu}$ as:

$$
\dot{\nu} = M(q)^{-1} (B\tau - h(q, \nu) + J^T(q) f)
$$

where:
- $M(q) \in \mathbb{R} ^{6+n \times 6+n}$ is the mass matrix
- $h(q, \nu) \in \mathbb{R} ^{6+n}$ is the bias forces term
- $B \in \mathbb{R}^{6+n \times n}$ is the input matrix
- $\tau \in \mathbb{R}^n$ is the input torque
- $J(q) \in \mathbb{R}^{6+n \times n \times 6}$ is the Jacobian of the contact points
- $f \in \mathbb{R}^{6+n}$ is the contact force.

Let's now compute the necessary quantities for the forward dynamics of ErgoCub model.

In [ ]:
# Compute the free floating mass matrix.
M = js.model.free_floating_mass_matrix(model=model, data=data)

# Compute the generalized free floating Jacobian matrix.
J = js.model.generalized_free_floating_jacobian(model=model, data=data)

# Compute the free floating bias forces vector.
h = js.model.free_floating_bias_forces(model=model, data=data)

# Extract the link forces from the reference object.
f_ext = references.link_forces(model=model, data=data)

# Extract the random joint forces from the reference object.
τ = references.joint_force_references(model=model)

# Create a dummy selection matrix.
B = jnp.block([jnp.zeros(shape=(model.dofs(), 6)), jnp.eye(model.dofs())]).T

We can now finally compute the acceleration of the robot:

In [ ]:
ν̇ = jnp.linalg.inv(M) @ (B @ τ - h + jnp.einsum("l6g,l6->g", J, f_ext))

print(f"Free floating acceleration: \n{ν̇ }")

We can obtain the same result by using the `jaxsim.api.model` module:

In [ ]:
ν̇ = jnp.concatenate(
    js.model.forward_dynamics(
        model=model,
        data=data,
        link_forces=references.link_forces(model=model, data=data),
        joint_forces=references.joint_force_references(model=model),
    )
)

print(f"Free floating acceleration: \n{ν̇ }")

**Note:**  
- `js.model.forward_dynamics` returns a tuple of the base acceleration and the joint accelerations, that's why we use `jnp.concatenate` to concatenate them.
- We used `jnp.einsum` to compute $J ^\top f$ instead of flattening the Jacobian and using `jnp.dot`.

Other interesting quantities that may be computed are the standalone Coriolis matrix, the centroidal Jacobian matrix and the generalized free floating Jacobian derivative.

In [ ]:
# Compute the standalone Coriolis matrix.
C = js.model.free_floating_coriolis_matrix(model=model, data=data)

# Compute the average centroidal velocity Jacobian.
G_J = js.model.average_velocity_jacobian(model=model, data=data)

# Compute the generalized free floating Jacobian derivative.
J̇ = js.model.generalized_free_floating_jacobian_derivative(model=model, data=data)

**Note:**
- These last methods are particularly computationally expensive and should be used with caution.

## Forward Kinematics

It is possible to compute the *forward kinematics* of the robot and then extract a link's transform by using its index:

In [ ]:
# Find the link index
foot_index = js.link.name_to_idx(model=model, link_name="l_ankle_2")

# Extract the link's transform
H_L = js.model.forward_kinematics(model=model, data=data)[foot_index]

print(f"Link transform: \n{H_L}")

or by directly using the `jaxsim.api.link` module:

In [ ]:
H_L = js.link.transform(
    model=model,
    data=data,
    link_index=foot_index,
)

print(f"Link transform: \n{H_L}")

### Computing the link's velocity

It is possible to compute the link's velocity by leveraging:

$$
v _L = J_L \nu
$$
    
where $J_L$ is the Jacobian of the link and $\nu$ is the generalized velocity.

For the link's Jacobian, we can either extract it from the generalized Jacobian as:

In [ ]:
# Extract the link's Jacobian from the generalized Jacobian
J_L = J[foot_index]

print(f"Link Jacobian: \n{J_L}")

or by using the `jaxsim.api.link` module:

In [ ]:
# Compute the link's Jacobian by using the `js.link` module
J_L = js.link.jacobian(model=model, data=data, link_index=foot_index)

print(f"Link Jacobian: \n{J_L}")

We can then compute the link's velocity as:

In [ ]:
# Extract the floating-base velocity from the data object
ν = data.generalized_velocity()

# Compute the link's velocity from the generalized velocity
v_L = J_L @ ν

print(f"Link Velocity: \n{v_L}")

or by directly using the `jaxsim.api.link`:

In [ ]:
# Compute the link's velocity by using the `js.link` module
v_L = js.link.velocity(model=model, data=data, link_index=foot_index)

print(f"Link Velocity: \n{v_L}")

## Contact Dynamics

### Contact Points Acceleration

The contact point acceleration can be calculated as the time derivative of their velocity:

$$
\frac{\partial}{\partial t} \dot{p} = \frac{\partial}{\partial t} \left( J _C \nu \right) = \frac{\partial J _C}{\partial t} \nu + J _C \frac{\partial \nu}{\partial t}
$$

which when brings to the following formulation:

$$
\ddot{p} = \dot{J _C}\nu + J _C \dot{\nu}
$$

The required quantities can be easily computed using the `jaxsim.api.contact` module.

In [ ]:
# Compute the Jacobian of the contact points
J_C = js.contact.jacobian(model=model, data=data)

# Compute the Jacobian derivative of the contact points
J̇_C = js.contact.jacobian_derivative(model=model, data=data)

# Extract the floating-base velocity from the data object
ν = data.generalized_velocity()

We already computed $\nu$ and $\dot{\nu}$ from the previous example, so we can now compute the contact points' acceleration:

In [ ]:
a = J̇_C @ ν + J_C @ ν̇

print(f"Contact points acceleration: \n{a}")

Other useful quantities that can be computed by using the `jaxsim.api.contact` module.

In [ ]:
# Compute the contact forces.
F_C, _ = js.contact.collidable_point_dynamics(
    model=model,
    data=data,
    link_forces=references.link_forces(model=model, data=data),
    joint_force_references=references.joint_force_references(model=model),
)

# Compute the contact points positions.
p_C = js.contact.collidable_point_positions(model=model, data=data)

# Compute the contact points velocities.
v_C = js.contact.collidable_point_velocities(model=model, data=data)